In [9]:
# !/usr/bin/python
# -*- coding: utf-8 -*-
# Author: Selvaria
# 对库内企业进行评分

import pymysql
from DBUtils.PooledDB import PooledDB
import pandas


_dbservice_pool = None

class MysqlPool(object): #设置数据库连接池和初始化    
    """
        MYSQL数据库对象，负责产生数据库连接 , 此类中的连接采用连接池实现
        获取连接对象：conn = Mysql.get_connection()
        释放连接对象;conn.close()或del conn
    """
    
    def __init__(self,mincached=5, maxcached=14,
                maxconnections=244, blocking=True, maxshared=0):
        """
            生成MySQL数据库连接池
        :param mincached: 最少的空闲连接数，如果空闲连接数小于这个数，pool会创建一个新的连接
        :param maxcached: 最大的空闲连接数，如果空闲连接数大于这个数，pool会关闭空闲连接
        :param maxconnections: 最大的连接数
        :param blocking: 当连接数达到最大的连接数时，在请求连接的时候，如果这个值是True，请求连接的程序会一直等待，
                          直到当前连接数小于最大连接数，如果这个值是False，会报错，
        :param maxshared: 当连接数达到这个数，新请求的连接会分享已经分配出去的连接
        """        
        db_config = {
            "host": '47.92.25.70',
            "port": 3306,
            "user": 'root',
            "passwd": 'Wfn031641',
            "db": 'zhaoshang',
            "charset": 'utf8'
        }
        self.pool = PooledDB(pymysql, mincached=mincached, maxcached=maxcached, maxconnections=maxconnections,
                             blocking=blocking, maxshared=maxshared, **db_config)

    def get_connection(self):
        return self.pool.connection()

    def close(self):
        self.pool.close()

    def __del__(self):
        self.close()

# 获取mysql数据库服务器的连接
def get_dbservice_mysql_conn(): #实例化连接池类，方便调用
    """
    :return: Object  MySQL Connection
    """
    global _dbservice_pool
    if not _dbservice_pool or not isinstance(_dbservice_pool, MysqlPool):
        _dbservice_pool = MysqlPool()
    return _dbservice_pool.get_connection()


#评分标准类
class CompanyScore(object):
    
    connection = get_dbservice_mysql_conn()
    
    def __init__(self, com_id):
        self.com_id = com_id
        
    def type_score(self): #根据企业类型的评分
        #connection = get_dbservice_mysql_conn()
        with self.connection.cursor() as cursor: 
            sql = "select `com_type` from `dic_company` where `q_id` = '%s'" % self.com_id
            cursor.execute(sql)
            type_name = cursor.fetchone()[0]
        if type_name is None:
            t_score = 0
        elif '有限责任' in type_name:
            t_score = 7
        elif '股份有限' in type_name:
            t_score = 5
        else:
            t_score = 2
        return t_score
    
    def industry_score(self): #根据企业行业评分
        with self.connection.cursor() as cursor: 
            sql = "select `industry` from `dic_company` where `q_id` = '%s'" % self.com_id
            cursor.execute(sql)
            industry_name = cursor.fetchone()[0]
        if industry_name is None:
            i_score = 0
        elif industry_name == '制造业':
            i_score = 6
        elif '批发和零售' in industry_name or '交通运输' in industry_name:
            i_score = 5
        elif  '采矿' in industry_name:
            i_score = 4
        else:
            i_score = 3
        return i_score
    
    def captital_score(self): #注册资本评分
        with self.connection.cursor() as cursor: 
            sql = "select `capital_num` from `dic_company` where `q_id` = '%s'" % self.com_id
            cursor.execute(sql)
            cap_num = cursor.fetchone()[0]
        if cap_num is None:
            cap_score = 0 
        elif cap_num < 300000:
            cap_score = 1
        elif cap_num >= 300000 and cap_num < 1000000:
            cap_score = 2
        elif cap_num >= 1000000 and cap_num < 5000000:
            cap_score = 4
        elif cap_num >= 5000000 and cap_num < 10000000:
            cap_score = 6
        elif cap_num >= 10000000 and cap_num < 100000000: #注册资金千万级
            cap_score = 8
        elif cap_num >= 100000000: #1亿以上的企业有1728个
            cap_score = 10
        return cap_score
    
    def patent_score(self): #知识产权评分
        with self.connection.cursor() as cursor: 
            sql = "SELECT distinct n.patent_num FROM dic_company m join com_patent n WHERE m.q_id = '{}' AND m.com = n.com_name".format(self.com_id)
            cursor.execute(sql)
            patent_list = cursor.fetchall()
        patent_num = len(patent_list)
        if patent_num > 7 and patent_num < 15:
            patent_score = 10
        elif patent_num >= 15:
            patent_score = 12
        else:
            patent_score = patent_num
        return patent_score
    
    def invest_score(self): #企业投资评分
        with self.connection.cursor() as cursor: 
            sql = "SELECT com_name FROM com_invest WHERE invest_com_id = '{}'".format(self.com_id)
            cursor.execute(sql)
            invest_list = cursor.fetchall()
        invest_num = len(invest_list)
        if invest_num > 0:
            invest_score = 4
        else:
            invest_score = 0
        return invest_score
    
    def finacial_score(self): #财务信息评分
        finacial_table = ['com_f_clean_asset','com_f_clean_margin','com_f_clean_profit_rate','com_f_gross_profit','com_f_revenue','com_f_total_asset']
        finacial_score = 0
        for table_name in finacial_table:
            with self.connection.cursor() as cursor: 
                sql = "SELECT `value` FROM `{0}` WHERE `q_id` = '{1}'".format(table_name, self.com_id)
                #print(sql)
                cursor.execute(sql)
                finacial_result = cursor.fetchall()
                #print(finacial_result)
                if len(finacial_result) == 0:
                    single_score = 0
                else:
                    single_score = 1
                finacial_score += single_score
        if finacial_score == 6:
            finacial_score = 8
        return finacial_score
    
    def final_score(self):
        score_list = [50]
        t_score = self.type_score()                            
        i_score = self.industry_score()
        c_score = self.captital_score()
        t_score = self.invest_score()
        p_score = self.patent_score()
        f_score = self.finacial_score()
        score_list.extend([t_score, i_score, c_score, t_score, p_score])
        #print(score_list)
        base_score = sum(score_list)
        if f_score == 8:
            final_score = base_score * 1.2
        elif f_score > 0 and f_score < 6:
            final_score = base_score * 1.1
        else:
            final_score = base_score
        return final_score
    
    
def test():
    #26ed4a801af5b9e9de6e8f724b9ade60 中石油山东
    #f04be507fc6da997f83a3a0a8d03d1fd 注册资本最多 3亿
    #5bad536fc41caf4357e1a58e0f277edf 有专利
    #78d80cac62dca925c237d184edcbc65d 有财务信息
    cs = CompanyScore('78d80cac62dca925c237d184edcbc65d') 
    #t_score = cs.type_score()                            
    #i_score = cs.industry_score()
    #c_score = cs.captital_score()
    #t_score = cs.invest_score()
    #p_score = cs.patent_score()
    f_score = cs.finacial_score()
    print(f_score)
    
def main(q_id):
    cs = CompanyScore(q_id)
    score = cs.final_score()
    return score
    
if __name__ == "__main__":
    #test()
    s = main('f04be507fc6da997f83a3a0a8d03d1fd')
    print(round(s,2))

94.6


In [3]:
import pandas

connection = get_dbservice_mysql_conn()
sql = "select `com`, `q_id` from `dic_company`"
df = pandas.read_sql(sql, connection)
df.head()

,com,q_id
0,山东墨龙石油机械股份有限公司,2abdc94b3d8f15aa7ed4faae599f87a8
1,山东省石油化工有限公司,a4535e9fc204bf9c15c98de2ddfce34e
2,山东中熬石油燃气有限公司,74334cac9933ab4d52dcf8979aa3e92a
3,山东石油天然气股份有限公司,01ea0f31cdf66024e8a7d26a26190547
4,中海日升石油有限公司,a25349bfcf5a1009c1ba234e3b232f9a


In [13]:
for index in range(20992,len(df)):
    q_id = df.loc[index, 'q_id']
    try:
        com_score = round(main(q_id),2)    
        with connection.cursor() as cursor: 
            sql_k = "insert into `dic_company_score_full`(`com_id`,`score`) values(%s,%s)" 
            cursor.execute(sql_k,(q_id, str(com_score)))
            connection.commit()    
    except pymysql.err.IntegrityError:
        continue
    except Exception as e:
        print(str(e))
        continue
    print("Now index:", index)

Now index: 20992
Now index: 20993
Now index: 20994
Now index: 20995
Now index: 20996
Now index: 20997
Now index: 20998
Now index: 20999
Now index: 21000
Now index: 21001
Now index: 21002
Now index: 21003
Now index: 21004
Now index: 21005
Now index: 21006
Now index: 21007
Now index: 21008
Now index: 21009
Now index: 21010
Now index: 21011
Now index: 21012
Now index: 21013
Now index: 21014
Now index: 21015
Now index: 21016
Now index: 21017
Now index: 21018
Now index: 21019
Now index: 21020
Now index: 21021
Now index: 21022
Now index: 21023
Now index: 21024
Now index: 21025
Now index: 21026
Now index: 21027
Now index: 21028
Now index: 21029
Now index: 21030
Now index: 21031
Now index: 21032
Now index: 21033
Now index: 21034
Now index: 21035
Now index: 21036
Now index: 21037
Now index: 21038
Now index: 21039
Now index: 21040
Now index: 21041
Now index: 21042
Now index: 21043
Now index: 21044
Now index: 21045
Now index: 21046
Now index: 21047
Now index: 21048
Now index: 21049
Now index: 210

Now index: 21474
Now index: 21475
Now index: 21476
Now index: 21477
Now index: 21478
Now index: 21479
Now index: 21480
Now index: 21481
Now index: 21482
Now index: 21483
Now index: 21484
Now index: 21485
Now index: 21486
Now index: 21487
Now index: 21488
Now index: 21489
Now index: 21490
Now index: 21491
Now index: 21492
Now index: 21493
Now index: 21494
Now index: 21495
Now index: 21496
Now index: 21497
Now index: 21498
Now index: 21499
Now index: 21500
Now index: 21501
Now index: 21502
Now index: 21503
Now index: 21504
Now index: 21505
Now index: 21506
Now index: 21507
Now index: 21508
Now index: 21509
Now index: 21510
Now index: 21511
Now index: 21512
Now index: 21513
Now index: 21514
Now index: 21515
Now index: 21516
Now index: 21517
Now index: 21518
Now index: 21519
Now index: 21520
Now index: 21521
Now index: 21522
Now index: 21523
Now index: 21524
Now index: 21525
Now index: 21526
Now index: 21527
Now index: 21528
Now index: 21529
Now index: 21530
Now index: 21531
Now index: 215

Now index: 21956
Now index: 21957
Now index: 21958
Now index: 21959
Now index: 21960
Now index: 21961
Now index: 21962
Now index: 21963
Now index: 21964
Now index: 21965
Now index: 21966
Now index: 21967
Now index: 21968
Now index: 21969
Now index: 21970
Now index: 21971
Now index: 21972
Now index: 21973
Now index: 21974
Now index: 21975
Now index: 21976
Now index: 21977
Now index: 21978
Now index: 21979
Now index: 21980
Now index: 21981
Now index: 21982
Now index: 21983
Now index: 21984
Now index: 21985
Now index: 21986
Now index: 21987
Now index: 21988
Now index: 21989
Now index: 21990
Now index: 21991
Now index: 21992
Now index: 21993
Now index: 21994
Now index: 21995
Now index: 21996
Now index: 21997
Now index: 21998
Now index: 21999
Now index: 22000
Now index: 22001
Now index: 22002
Now index: 22003
Now index: 22004
Now index: 22005
Now index: 22006
Now index: 22007
Now index: 22008
Now index: 22009
Now index: 22010
Now index: 22011
Now index: 22012
Now index: 22013
Now index: 220

Now index: 22438
Now index: 22439
Now index: 22440
Now index: 22441
Now index: 22442
Now index: 22443
Now index: 22444
Now index: 22445
Now index: 22446
Now index: 22447
Now index: 22448
Now index: 22449
Now index: 22450
Now index: 22451
Now index: 22452
Now index: 22453
Now index: 22454
Now index: 22455
Now index: 22456
Now index: 22457
Now index: 22458
Now index: 22459
Now index: 22460
Now index: 22461
Now index: 22462
Now index: 22463
Now index: 22464
Now index: 22465
Now index: 22466
Now index: 22467
Now index: 22468
Now index: 22469
Now index: 22470
Now index: 22471
Now index: 22472
Now index: 22473
Now index: 22474
Now index: 22475
Now index: 22476
Now index: 22477
Now index: 22478
Now index: 22479
Now index: 22480
Now index: 22481
Now index: 22482
Now index: 22483
Now index: 22484
Now index: 22485
Now index: 22486
Now index: 22487
Now index: 22488
Now index: 22489
Now index: 22490
Now index: 22491
Now index: 22492
Now index: 22493
Now index: 22494
Now index: 22495
Now index: 224

Now index: 22920
Now index: 22921
Now index: 22922
Now index: 22923
Now index: 22924
Now index: 22925
Now index: 22926
Now index: 22927
Now index: 22928
Now index: 22929
Now index: 22930
Now index: 22931
Now index: 22932
Now index: 22933
Now index: 22934
Now index: 22935
Now index: 22936
Now index: 22937
Now index: 22938
Now index: 22939
Now index: 22940
Now index: 22941
Now index: 22942
Now index: 22943
Now index: 22944
Now index: 22945
Now index: 22946
Now index: 22947
Now index: 22948
Now index: 22949
Now index: 22950
Now index: 22951
Now index: 22952
Now index: 22953
Now index: 22954
Now index: 22955
Now index: 22956
Now index: 22957
Now index: 22958
Now index: 22959
Now index: 22960
Now index: 22961
Now index: 22962
Now index: 22963
Now index: 22964
Now index: 22965
Now index: 22966
Now index: 22967
Now index: 22968
Now index: 22969
Now index: 22970
Now index: 22971
Now index: 22972
Now index: 22973
Now index: 22974
Now index: 22975
Now index: 22976
Now index: 22977
Now index: 229

...

Now index: 96561
Now index: 96562
Now index: 96563
Now index: 96564
Now index: 96565
Now index: 96566
Now index: 96567
Now index: 96568
Now index: 96569
Now index: 96570
Now index: 96571
Now index: 96572
Now index: 96573
Now index: 96574
Now index: 96575
Now index: 96576
Now index: 96577
Now index: 96578
Now index: 96579
Now index: 96580
Now index: 96581
Now index: 96582
Now index: 96583
Now index: 96584
Now index: 96585
Now index: 96586
Now index: 96587
Now index: 96588
Now index: 96589
Now index: 96590
Now index: 96591
Now index: 96592
Now index: 96593
Now index: 96594
Now index: 96595
Now index: 96596
Now index: 96597
Now index: 96598
Now index: 96599
Now index: 96600
Now index: 96601
Now index: 96602
Now index: 96603
Now index: 96604
Now index: 96605
Now index: 96606
Now index: 96607
Now index: 96608
Now index: 96609
Now index: 96610
Now index: 96611
Now index: 96612
Now index: 96613
Now index: 96614
Now index: 96615
Now index: 96616
Now index: 96617
Now index: 96618
Now index: 966

Now index: 97044
Now index: 97045
Now index: 97046
Now index: 97047
Now index: 97048
Now index: 97049
Now index: 97050
Now index: 97051
Now index: 97052
Now index: 97053
Now index: 97054
Now index: 97055
Now index: 97056
Now index: 97057
Now index: 97058
Now index: 97059
Now index: 97060
Now index: 97061
Now index: 97062
Now index: 97063
Now index: 97064
Now index: 97065
Now index: 97066
Now index: 97067
Now index: 97068
Now index: 97069
Now index: 97070
Now index: 97071
Now index: 97072
Now index: 97073
Now index: 97074
Now index: 97075
Now index: 97076
Now index: 97077
Now index: 97078
Now index: 97079
Now index: 97080
Now index: 97081
Now index: 97082
Now index: 97083
Now index: 97084
Now index: 97085
Now index: 97086
Now index: 97087
Now index: 97088
Now index: 97089
Now index: 97090
Now index: 97091
Now index: 97092
Now index: 97093
Now index: 97094
Now index: 97095
Now index: 97096
Now index: 97097
Now index: 97098
Now index: 97099
Now index: 97100
Now index: 97101
Now index: 971

Now index: 97533
Now index: 97534
Now index: 97535
Now index: 97536
Now index: 97537
Now index: 97538
Now index: 97539
Now index: 97540
Now index: 97541
Now index: 97542
Now index: 97543
Now index: 97544
Now index: 97545
Now index: 97546
Now index: 97547
Now index: 97548
Now index: 97549
Now index: 97550
Now index: 97551
Now index: 97552
Now index: 97553
Now index: 97554
Now index: 97555
Now index: 97556
Now index: 97557
Now index: 97558
Now index: 97559
Now index: 97560
Now index: 97561
Now index: 97562
Now index: 97563
Now index: 97564
Now index: 97565
Now index: 97566
Now index: 97567
Now index: 97568
Now index: 97569
Now index: 97570
Now index: 97571
Now index: 97572
Now index: 97573
Now index: 97574
Now index: 97575
Now index: 97577
Now index: 97578
Now index: 97579
Now index: 97580
Now index: 97581
Now index: 97582
Now index: 97583
Now index: 97584
Now index: 97585
Now index: 97586
Now index: 97587
Now index: 97588
Now index: 97589
Now index: 97590
Now index: 97591
Now index: 975

Now index: 98020
Now index: 98021
Now index: 98022
Now index: 98023
Now index: 98024
Now index: 98025
Now index: 98026
Now index: 98027
Now index: 98028
Now index: 98029
Now index: 98030
Now index: 98031
Now index: 98032
Now index: 98033
Now index: 98034
Now index: 98035
Now index: 98036
Now index: 98037
Now index: 98038
Now index: 98039
Now index: 98040
Now index: 98041
Now index: 98042
Now index: 98043
Now index: 98044
Now index: 98045
Now index: 98046
Now index: 98047
Now index: 98048
Now index: 98049
Now index: 98050
Now index: 98051
Now index: 98052
Now index: 98053
Now index: 98054
Now index: 98055
Now index: 98056
Now index: 98057
Now index: 98058
Now index: 98059
Now index: 98060
Now index: 98061
Now index: 98062
Now index: 98063
Now index: 98064
Now index: 98065
Now index: 98066
Now index: 98067
Now index: 98068
Now index: 98069
Now index: 98070
Now index: 98071
Now index: 98072
Now index: 98073
Now index: 98074
Now index: 98075
Now index: 98076
Now index: 98077
Now index: 980

Now index: 98506
Now index: 98507
Now index: 98508
Now index: 98509
Now index: 98510
Now index: 98511
Now index: 98513
Now index: 98514
Now index: 98515
Now index: 98516
Now index: 98517
Now index: 98518
Now index: 98519
Now index: 98520
Now index: 98521
Now index: 98522
Now index: 98523
Now index: 98524
Now index: 98525
Now index: 98526
Now index: 98527
Now index: 98528
Now index: 98529
Now index: 98530
Now index: 98531
Now index: 98532
Now index: 98533
Now index: 98534
Now index: 98535
Now index: 98536
Now index: 98537
Now index: 98538
Now index: 98539
Now index: 98540
Now index: 98541
Now index: 98542
Now index: 98543
Now index: 98544
Now index: 98545
Now index: 98546
Now index: 98547
Now index: 98548
Now index: 98549
Now index: 98550
Now index: 98551
Now index: 98552
Now index: 98553
Now index: 98554
Now index: 98555
Now index: 98556
Now index: 98557
Now index: 98558
Now index: 98559
Now index: 98560
Now index: 98561
Now index: 98562
Now index: 98563
Now index: 98564
Now index: 985

In [17]:
import pandas
import seaborn

connection = get_dbservice_mysql_conn()
sql = "select * from `dbscan_all_digital` where capital_num > 100000000"
df = pandas.read_sql(sql, connection)
df.head()

,id,capital_num,insured,province,city,person,com_type,industry,member,dbscan_code
0,1,797848384,1166,10,266,19238,160,7,0,-1
1,2,130000000,133,10,266,40864,69,13,2,-1
2,4,359264000,37,10,266,9069,153,18,0,-1
3,6,150000000,0,10,266,37802,153,7,5,-1
4,10,1170000000,0,10,266,28464,69,2,0,-1


In [18]:
print(len(df))

1728


In [14]:
df_c = df['capital_num']
df_c.describe() #中位数15W，最高的25%位500W，1亿以上的有1728个

count    9.888600e+04
mean     1.509247e+07
std      1.061462e+08
min      0.000000e+00
25%      0.000000e+00
50%      1.500000e+05
75%      5.000000e+06
max      2.147484e+09
Name: capital_num, dtype: float64

In [24]:
connection = get_dbservice_mysql_conn()
with connection.cursor() as cursor: 
    sql = "SELECT distinct n.patent_num FROM dic_company m join com_patent n WHERE m.q_id = '26ed4a801af5b9e9de6e8f724b9ade60' AND m.com = n.com_name"
    cursor.execute(sql)
    cap_num = cursor.fetchall()
print(len(cap_num))

0
